In [1]:
import numpy as np
try:
    import pyaudio
except:
    !pip3 install pyaudio
    import pyaudio

import wave, time, struct

from scipy import signal
from scipy.fft import fft, fftfreq, fftshift, ifft
from scipy.signal import blackman
from scipy.io.wavfile import write

try:
    import librosa, librosa.display
except:
    !pip3 install librosa
    import librosa, librosa.display
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = [7.50, 3.50]
plt.rcParams["figure.autolayout"] = True

def nicegrid(ax=plt):
    ax.grid(True, which='major', color='#666666', linestyle=':')
    ax.grid(True, which='minor', color='#999999', linestyle=':', alpha=0.2)
    ax.minorticks_on()

In [2]:
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
CHUNK = 512
RECORD_SECONDS = 10
WAVE_OUTPUT_FILENAME = "data/example_recordedFile.wav"
audio = pyaudio.PyAudio()

In [3]:
print("----------------------record device list---------------------")
info = audio.get_host_api_info_by_index(0)
numdevices = info.get('deviceCount')
for i in range(0, numdevices):
        if (audio.get_device_info_by_host_api_device_index(0, i).get('maxInputChannels')) > 0:
            print("Input Device id ", i, " - ", audio.get_device_info_by_host_api_device_index(0, i).get('name'))

print("-------------------------------------------------------------")

----------------------record device list---------------------
Input Device id  0  -  MacBook Pro Microphone
Input Device id  2  -  Microsoft Teams Audio
-------------------------------------------------------------


In [4]:
#index = int(input())
index = 5
print("recording via index "+str(index))

recording via index 5


In [ ]:
stream = audio.open(format=FORMAT, channels=CHANNELS,
                rate=RATE, input=True, input_device_index = index,
                frames_per_buffer=CHUNK)
print ("recording started")
Recordframes = []

start = time.time()
for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK, exception_on_overflow = False)
    Recordframes.append(data)
    if (i%100 == 0):
        print(f"{time.time() - start:.1f} ",  end = '')
tf = time.time() - start
print ("\nrecording stopped")

recorded_signal = np.hstack(Recordframes)

stream.stop_stream()
stream.close()
audio.terminate()

In [ ]:
waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
waveFile.setnchannels(CHANNELS)
waveFile.setsampwidth(audio.get_sample_size(FORMAT))
waveFile.setframerate(RATE)
waveFile.writeframes(b''.join(Recordframes))
waveFile.close()

In [ ]:
recorded_signal_f = np.frombuffer(recorded_signal, dtype='int16').astype('float32')
plt.figure(figsize=(20, 5))
t_ignore = 1
librosa.display.waveshow(recorded_signal_f[t_ignore*RATE:], sr=RATE)
plt.title('Waveplot', fontdict=dict(size=18))
plt.xlabel('Time', fontdict=dict(size=18))
plt.ylabel('Amplitude', fontdict=dict(size=18))
plt.xlim(0,tf-t_ignore)
plt.savefig('./data/mic_recorder_0.png', bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
y = recorded_signal_f[t_ignore*RATE:]
y = y/max(y)
N = len(y)
fs = RATE
T = N/fs
t = np.arange(N)*T/N

In [ ]:
Y_fft = fft(y)
f_bl = fftfreq(N, 1/fs)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(8,4), dpi=150)
fig.tight_layout(pad=2)
ax1.plot(t, y.real, 'b:', t, y.imag, 'r:')
ax1.set_ylabel('$y_{bl}[nT_s]$')
ax1.set_xlabel('$t$')
ax2.set_xlim([0, 19])
ax1.legend(['y(t).real','y(t).imag'])
ax1.set_title('Snippet of data')
nicegrid(ax1)

ax2.loglog(f_bl[0:N//2], np.abs(Y_fft[0:N//2]),'b-')
ax2.set_ylabel('FFT $y_{bl}[nT_s]$')
ax2.set_xlabel('$f$')
ax2.set_xlim([10, 10000])
ax2.set_ylim([.1, 2000])
ax2.legend(['Y_{fft}(f)','Y_{given}(f)'])
ax2.set_title('FFT of data')
plt.show()

In [ ]:
# bounds of noise in Y(f)
f_low = 200
f_high = 20000

# conversion from frequency to index count 
delta_freq = fs/N

# find equivalent positions in Y(f) 
n_low = int(f_low/delta_freq)
n_high = int(f_high/delta_freq)
L1 = np.arange(n_low,n_high) # noise indices

# create noise vector
Y_n = np.zeros((N,), dtype=complex)
# create random phase needed for correct ifft process
rng = np.random.default_rng()
phi = rng.uniform(0, 2*np.pi, (len(L1),))

# create Y(f)
if (0): # half-sine shape noise
    mag_temp = np.sin(np.pi*np.arange(0,n_high-n_low)/(n_high-n_low))
else: # rect noise
    mag_temp = 850*np.ones([1,n_high-n_low])
    
Y_n[L1] = mag_temp*np.exp(1j*phi)
# add complex conjugate at upper end to ensure that y[n] real
Y_n[N-L1] = np.conj(Y_n[L1])

# create y[n]
noise = ifft(Y_n)

In [ ]:
yn = y + noise.real
Y_fft_n = fft(yn)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(8,4), dpi=150)
fig.tight_layout(pad=2)
ax1.plot(t, y, 'b:', t, yn, 'r:',alpha=0.2)
ax1.set_ylabel('$y_{bl}[nT_s]$')
ax1.set_xlabel('$t$')
ax1.legend(['Signal','Noisy signal'])
ax2.set_xlim([0, 19])
ax1.set_title('Snippet of data')
nicegrid(ax1)

ax2.loglog(f_bl[0:N//2], np.abs(Y_fft[0:N//2]),'b-',alpha=0.6)
ax2.loglog(f_bl[0:N//2], np.abs(Y_fft_n[0:N//2]),'r:',alpha=0.3)
ax2.set_ylabel('FFT $y_{bl}[nT_s]$')
ax2.set_xlabel('$f$')
ax2.set_xlim([10, 5000])
ax2.set_ylim([10, 3000])
ax2.legend(['Signal','Noisy signal'])
ax2.set_title('FFT of data')
nicegrid(ax2)
plt.show()

In [ ]:
# Low Pass
b_low, a_low = signal.butter(2, 200, 'low', fs=fs)
f_low, h_low = signal.freqz(b_low, a_low,fs=fs,worN=1000)
yn_lp = signal.lfilter(b_low, a_low, yn)
Yn_lp_fft = fft(yn_lp)

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize=(8,4), dpi=150)
fig.tight_layout(pad=2)
eps = 1e-12
ax1.loglog(f_low, abs(h_low+eps),'b-',label='LP')
ax1.set_title('Butterworth filter frequency response')
ax1.set_xlabel('Frequency [Hz]')
ax1.set_ylabel('Amplitude')
ax1.set_xlim([10, 5000])
ax1.set_ylim([.001, 2])
nicegrid()

ax2.loglog(f_bl[0:N//2], np.abs(Y_fft[0:N//2]),'b-',alpha=0.6)
ax2.loglog(f_bl[0:N//2], np.abs(Y_fft_n[0:N//2]),'r:',alpha=0.4)
ax2.loglog(f_bl[0:N//2], np.abs(Yn_lp_fft[0:N//2]),'g--',alpha = .8)
ax2.set_ylabel('FFT $y_{bl}[nT_s]$')
ax2.set_xlabel('$f$')
ax2.set_xlim([10, 5000])
ax2.set_ylim([10, 3000])
ax2.legend(['FFT sig','FFT sig+n','LP of sig+n'])
ax2.set_title('FFT of data')
plt.show()

In [ ]:
# save the data
y_norm = y/max(abs(y))
write("data/example.wav", RATE, y_norm.astype(np.float32))
yn_norm = yn/max(abs(yn))
write("data/example_noisy.wav", RATE, yn_norm.astype(np.float32))
yn_lp_norm = yn/max(abs(yn_lp))
write("data/example_noisy_lp.wav", RATE, yn_lp_norm.astype(np.float32))

## Modulation

In [ ]:
# Develop 3 carriers to modulate the signal
f_carrier_1 = 5000
f_carrier_2 = 2*f_carrier_1
f_carrier_3 = 3*f_carrier_1

II = np.arange(0,N//3)
N_t1 = len(II)
# Split the signal into 3 pieces, then modulate each one
t1 = t[II]
t2 = t[N//3+II]
t3 = t[2*(N//3)+II]

y_signal_1 = y[II]
y_signal_2 = y[N//3+II]
y_signal_3 = y[2*(N//3)+II]

y_carrier_1 = np.cos(2*np.pi*f_carrier_1*t1)
y_carrier_2 = np.cos(2*np.pi*f_carrier_2*t2)
y_carrier_3 = np.cos(2*np.pi*f_carrier_3*t3)

y_modulated = y_signal_1*y_carrier_1 
y_modulated += y_signal_2*y_carrier_2 
y_modulated += y_signal_3*y_carrier_3

# transform all of the signal pieces
Y_1 = fft(y_signal_1)
Y_2 = fft(y_signal_2)
Y_3 = fft(y_signal_3)
Y_modulated = fft(y_modulated)
f_modulated = fftfreq(N_t1, 1/fs)

In [ ]:
fig, (ax1) = plt.subplots(1,1, figsize=(8,4), dpi=150)
ax1.semilogy(f_modulated[0:N_t1//2], np.abs(Y_modulated[0:N_t1//2]),'b-',alpha=0.6)
ax1.set_ylabel('FFT $y_{bl}[nT_s]$')
ax1.set_xlabel('$f$')
ax1.set_xlim([1000, fs/2])
ax1.set_ylim([1, 3000]) 
ax1.legend(['FFT sig','FFT sig+n','LP of sig+n'])
ax1.set_title('FFT of modulated data')
plt.show()

## Demodulate Signal and low-pass filter

In [ ]:
f_voice_lp = 2000
bb_low, aa_low = signal.butter(2, f_voice_lp, 'low', fs=fs)

y_demodulated_1 = y_modulated*y_carrier_1 
y_demodulated_1_lp = signal.lfilter(bb_low, aa_low, y_demodulated_1)
Y_demodulated_1_lp = fft(y_demodulated_1_lp)

y_demodulated_2 = y_modulated*y_carrier_2 
y_demodulated_2_lp = signal.lfilter(bb_low, aa_low, y_demodulated_2)
Y_demodulated_2_lp = fft(y_demodulated_2_lp)

y_demodulated_3 = y_modulated*y_carrier_3 
y_demodulated_3_lp = signal.lfilter(bb_low, aa_low, y_demodulated_3)
Y_demodulated_3_lp = fft(y_demodulated_3_lp)

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(3,1, figsize=(8,8), dpi=150)
ax1.loglog(f_modulated[0:N_t1//2-1], np.abs(Y_1[0:N_t1//2-1]),'b-',alpha=1)
ax1.loglog(f_modulated[0:N_t1//2], np.abs(Y_demodulated_1_lp[0:N_t1//2]),'r-',alpha=0.4)
ax1.set_ylabel('FFT $y_{bl}[nT_s]$')
ax1.set_xlabel('$f$')
ax1.set_xlim([10, 2000])
ax1.set_ylim([1, 5000])
ax1.legend(['Modulated','Demod_1'])
ax1.set_title('FFT of demodulated data')

ax2.loglog(f_modulated[0:N_t1//2-1], np.abs(Y_2[0:N_t1//2-1]),'b-',alpha=1)
ax2.loglog(f_modulated[0:N_t1//2], np.abs(Y_demodulated_2_lp[0:N_t1//2]),'r-',alpha=0.4)
ax2.set_ylabel('FFT $y_{bl}[nT_s]$')
ax2.set_xlabel('$f$')
ax2.set_xlim([10, 2000])
ax2.set_ylim([1, 5000])
ax2.legend(['Modulated','Demod_2'])
ax2.set_title('FFT of demodulated data')

ax3.loglog(f_modulated[0:N_t1//2-1], np.abs(Y_3[0:N_t1//2-1]),'b-',alpha=1)
ax3.loglog(f_modulated[0:N_t1//2], np.abs(Y_demodulated_3_lp[0:N_t1//2]),'r-',alpha=0.4)
ax3.set_ylabel('FFT $y_{bl}[nT_s]$')
ax3.set_xlabel('$f$')
ax3.set_xlim([10, 2000])
ax3.set_ylim([1, 5000])
ax3.legend(['Modulated','Demod_3'])
ax3.set_title('FFT of demodulated data')
plt.show()

## Save the modulated and demodulated signals

In [ ]:
y_modulated_norm = y_modulated/max(abs(y_modulated))
write("data/example_modulated.wav", RATE, y_modulated_norm.astype(np.float32))

y_demodulated_1_lp_norm = y_demodulated_1_lp /max(abs(y_demodulated_1_lp))
write("data/example_demodulated_1_lp.wav", RATE, y_demodulated_1_lp_norm.astype(np.float32))

y_demodulated_2_lp_norm = y_demodulated_2_lp /max(abs(y_demodulated_2_lp))
write("data/example_demodulated_2_lp.wav", RATE, y_demodulated_2_lp_norm.astype(np.float32))

y_demodulated_3_lp_norm = y_demodulated_3_lp /max(abs(y_demodulated_3_lp))
write("data/example_demodulated_3_lp.wav", RATE, y_demodulated_3_lp_norm.astype(np.float32))

In [ ]:
len(y_modulated_norm)

In [ ]:
y_modulated_norm_long = y_modulated_norm
for ii in range(0,10):
    print(ii)
    y_modulated_norm_long = np.concatenate((y_modulated_norm_long,y_modulated_norm))

In [ ]:
write("data/example_modulated.wav", RATE, y_modulated_norm_long.astype(np.float32))

In [ ]:
len(y_modulated_norm_long)